In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%capture

!pip install tensorboardX

# dhash used for shot separation / frame skipping
!pip install dhash

In [0]:
!git clone https://github.com/simonedeldeo/Super-SloMo.git

Cloning into 'Super-SloMo'...
remote: Enumerating objects: 174, done.
remote: Total 174 (delta 0), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (174/174), 26.88 MiB | 21.09 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [0]:
%cd Super-SloMo

/content/Super-SloMo


In [0]:
# creaz dataset

!rm -rf sep_eps
!rm -rf ds
!mkdir ds

import cv2
import dhash
from PIL import Image
import os
import shutil

def get_num_bits_different(hash1, hash2):
    return bin(hash1 ^ hash2).count('1')

#vids = ['at_s01e01.mp4', 'at_s01e02.mp4', 'at_s01e03.mp4', 'at_s01e04.mp4', 'sao_e01.mkv', 'Akira.mkv']
vids = ['at_s01e01.mp4', 'sao_e01.mkv', 'Akira.mkv']

# used to save ds
fld = 0

for vid_path in vids:

  cap = cv2.VideoCapture('/content/drive/My Drive/__progetto int sys/data/' + vid_path)
  vid_name = vid_path.split('.')[0]
  totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  if totalFrames > 10000:
    totalFrames = 10000
  main_dir = 'sep_eps/' + vid_name
  if os.path.exists(main_dir):
      shutil.rmtree(main_dir)
  os.makedirs(main_dir)

  # 0 - 2: frame ripetuto => non lo salvo
  # 0 - 9 per Akira
  thresh_rip = 2
  if vid_name == 'Akira':
    thresh_rip = 9
    # skip 2400 frames
    for _ in range(2400):
      _, _ = cap.read()

  id_scena = 1
  ret, prec = cap.read()
  path = main_dir + '/' + str(id_scena).zfill(3)
  if not os.path.exists(path):
      os.mkdir(path)
  cv2.imwrite(path + '/0.jpg', prec)
  x = int(prec.shape[0] / 5)
  y = int(prec.shape[1] / 5)
  for i in range(1, int(totalFrames)):
      try:
          ret, frame = cap.read()
          prec_res = cv2.resize(prec, (y, x))
          frame_res = cv2.resize(frame, (y, x))
          h1 = dhash.dhash_int(Image.fromarray(prec_res), size=16)
          h2 = dhash.dhash_int(Image.fromarray(frame_res), size=16)

          bit_diff = get_num_bits_different(h1, h2)

          if bit_diff > thresh_rip:
              # threshold trovato sperimentalmente per sep shot = 213
              # usando 205, i falsi positivi aumentavano solo di 5
              # meglio qualche falso pos in piu' che un falso neg
              if bit_diff > 204:
                  id_scena += 1
              path = main_dir + '/' + str(id_scena).zfill(3)
              if not os.path.exists(path):
                  os.mkdir(path)
              frame = cv2.resize(frame, (1280, 720))
              cv2.imwrite(path + '/' + str(i) + '.jpg', frame)

          prec = frame

      except:
          print('frame finale: ', i)
  print(vid_name)

  #########################################################

  ep_path = 'sep_eps/' + vid_name
  scenes = os.listdir(ep_path)
  scenes.sort()

  da_canc = []
  for s in scenes:
      folder = os.path.join(ep_path, s)
      frames_number = os.listdir(folder)
      if len(frames_number)<12:
          da_canc.append(folder)

  for f in da_canc:
      shutil.rmtree(f)

  #########################################################

  twelveplets = []
  lista_dir = os.listdir(ep_path)
  lista_dir.sort()
  for d in lista_dir:
    dir_path = os.path.join(ep_path, d)
    toAppend = os.listdir(dir_path)
    if len(toAppend)>11:
      files = []
      for f in toAppend:
        files.append(os.path.join(dir_path, f))
      files.sort()
      for i in range(len(files)-11):
        tmp = []
        for j in range(12):
          tmp.append(files[i+j])
        twelveplets.append(tmp)
  print(len(twelveplets))

  #########################################################

  train_spl = int(len(twelveplets) * 9 / 10)
  val_spl = int(len(twelveplets) * 9.5 / 10)

  saved_tp = 0
  # save ds
  for t in twelveplets:
    if saved_tp < train_spl:
      save_dir = 'ds/train/' + str(fld).zfill(7)
    elif saved_tp < val_spl:
      save_dir = 'ds/validation/' + str(fld).zfill(7)
    else:
      save_dir = 'ds/test/' + str(fld).zfill(7)
    os.makedirs(save_dir)
    for i in range(12):
      shutil.copyfile(t[i], save_dir + '/frame' + str(i).zfill(2) + '.jpg')
    fld += 1
    saved_tp += 1
    
  print(ep_path)
  shutil.rmtree(ep_path)

at_s01e01
4601
sep_eps/at_s01e01
sao_e01
5147
sep_eps/sao_e01
Akira
5456
sep_eps/Akira


In [0]:
!ls ds/train | wc -l
!ls ds/validation | wc -l
!ls ds/test | wc -l
!du -hs ds

13682
760
762
32G	ds


In [0]:
#!cp '/content/drive/My Drive/__progetto int sys/ckps/SuperSloMo.ckpt' 'data/SuperSloMo.ckpt'
!cp '/content/drive/My Drive/__progetto int sys/ckps/SuperSloMo/_new/AniDSfull720_10k/SuperSloMo0.ckpt' 'data/SuperSloMo.ckpt'

In [0]:
!rm -rf checkpoints
!mkdir checkpoints

In [0]:
!date
!python train.py --dataset_root ds --checkpoint_dir checkpoints --train_continue True --checkpoint data/SuperSloMo.ckpt --checkpoint_epoch 1 --epochs 3
!date

Sat Feb 15 11:53:11 UTC 2020
Dataset SuperSloMo
    Number of datapoints: 13682
    Root Location: ds/train
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=[0.429, 0.431, 0.397], std=[1, 1, 1])
                         )
 Dataset SuperSloMo
    Number of datapoints: 760
    Root Location: ds/validation
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=[0.429, 0.431, 0.397], std=[1, 1, 1])
                         )

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:08<00:00, 62.7MB/s]
Epoch:  1
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`

In [0]:
!mkdir '/content/drive/My Drive/__progetto int sys/ckps/SuperSloMo/_new/AniDSfull720_10k_mr/'
!cp /content/Super-SloMo/checkpoints/SuperSloMo0.ckpt '/content/drive/My Drive/__progetto int sys/ckps/SuperSloMo/_new/AniDSfull720_10k_mr/'
!cp /content/Super-SloMo/checkpoints/SuperSloMo1.ckpt '/content/drive/My Drive/__progetto int sys/ckps/SuperSloMo/_new/AniDSfull720_10k_mr/'